In [1]:
import pandas as pd
from var_names import *
# import numpy as np
import yaml

In [2]:
# read variables' categories file
def read_categories_from_file(file_path):
    """Read variables and their categories from a YAML file."""
    try:
        with open(file_path, 'r') as file:
            categories_dict = yaml.safe_load(file)
        return categories_dict
    except IOError as e:
        print(f"An error occurred: {e.strerror}")
    except yaml.YAMLError as exc:
        print(f"YAML error: {exc}")
    return {}

In [3]:
def frequency_distribution(series, categories):
    """Calculate value counts and percentages for a pandas Series including all possible categories."""
    # Convert the series to a categorical series with the specified categories
    cat_series = pd.Categorical(series, categories=categories)
    
    # Calculate frequency counts and percentages
    freq_count = pd.Series(cat_series).value_counts(dropna=False).sort_index()
    relative_freq = pd.Series(cat_series).value_counts(normalize=True, dropna=False).mul(100).round(2).sort_index()
    relative_freq_str = relative_freq.astype(str) + '%'
    
    # Combine into a DataFrame
    result = pd.DataFrame({
        'N': freq_count,
        '%': relative_freq_str
    })
    
    # Reindex the DataFrame to include all categories, filling with 0 for the counts and '0%' for the percentages
    result = result.reindex(categories, fill_value=0)
    result['%'] = result['%'].replace({'0.0%': '0%'})

    return result

In [4]:
# codebook = pd.read_excel('codebook.xlsx')
baseline = pd.read_csv('tba_data.csv')

In [5]:
# baseline

In [6]:
cat_dict = read_categories_from_file('var_cat.yaml')

In [7]:
cat_dict

{'age': ['under_18', '18-24', '25-55', 'over_55'],
 'consent': ['agree', 'disagree'],
 'gender': ['male', 'female', 'other'],
 'location': ['mersin', 'hatay', 'gaziantep', 'urfa'],
 'scale1': ['very_poor', 'poor', 'neither', 'good', 'very_good', 888, 999],
 'scale2': ['very_dissatisfied',
  'dissatisfied',
  'neither',
  'satisfied',
  'very_satisfied',
  888,
  999],
 'scale3': ['not_at_all',
  'a_little',
  'moderate_amount',
  'very_much',
  'extreme_amount',
  888,
  999],
 'scale4': ['not_at_all',
  'a_little',
  'moderately',
  'mostly',
  'completely',
  888,
  999],
 'scale5': ['never',
  'seldom',
  'quite_often',
  'very_often',
  'always',
  888,
  999],
 'scale6': ['none', 'very_few', 'some', 'many', 'very_many', 888, 999],
 'scale7': ['none',
  'a_little',
  'moderate_amount',
  'quite_a_lot',
  'very_much',
  888,
  999],
 'scale8': ['strongly_disagree',
  'disagree',
  'neither',
  'agree',
  'strongly_agree',
  888,
  999],
 'scale9': ['not_at_all',
  'a_little',
  'som

In [9]:
def tabulation(df, variable, cat):
   """Tabulate value counts for a specific variable in a DataFrame."""
   series = df[variable]
   frequencies = frequency_distribution(series=series, categories=cat)
   return frequencies

In [10]:
question = 'qual_life1'
var_cat = 'scale1'
categories = cat_dict.get(var_cat, [])
frequency = tabulation(baseline, question, categories)
frequency.to_clipboard()

In [11]:
frequency

,N,%
very_poor,17,16.04%
poor,28,26.42%
neither,32,30.19%
good,25,23.58%
very_good,4,3.77%
888,0,0%
999,0,0%


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
#    for var in question:
#       Check if the variable exists in the dataframe
#       if var in baseline.columns:
         # Apply your function to the variable, and get the results
         # result_df = normalized_value_counts(baseline[var])
         # Write each DataFrame to a specific sheet
         # result_df.to_excel(writer, sheet_name=var)
      # else:
      #    print(f"Variable {var} not found in DataFrame.")

In [ ]:
baseline['network_size'].value_counts()

In [ ]:
# baseline['relative_support2'].value_counts(normalize=True) * 100